<a href="https://colab.research.google.com/github/springboardmentor442n-coder/emotion_Detection/blob/Sarika-M/Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()   # select kaggle.json


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sarikamotamari","key":"2a10e086e7b3dea15eff5bc7c5ff52ab"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d msambare/fer2013


Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013
License(s): DbCL-1.0
  0% 0.00/60.3M [00:00<?, ?B/s]
100% 60.3M/60.3M [00:00<00:00, 1.72GB/s]


In [4]:
!unzip fer2013.zip


Streaming output truncated to the last 5000 lines.
  inflating: train/sad/Training_65242339.jpg  
  inflating: train/sad/Training_65267116.jpg  
  inflating: train/sad/Training_65275626.jpg  
  inflating: train/sad/Training_6529266.jpg  
  inflating: train/sad/Training_65329617.jpg  
  inflating: train/sad/Training_65338712.jpg  
  inflating: train/sad/Training_65338797.jpg  
  inflating: train/sad/Training_65387162.jpg  
  inflating: train/sad/Training_65404494.jpg  
  inflating: train/sad/Training_65426218.jpg  
  inflating: train/sad/Training_65430136.jpg  
  inflating: train/sad/Training_65437377.jpg  
  inflating: train/sad/Training_6545735.jpg  
  inflating: train/sad/Training_65463385.jpg  
  inflating: train/sad/Training_65473985.jpg  
  inflating: train/sad/Training_65502829.jpg  
  inflating: train/sad/Training_65505359.jpg  
  inflating: train/sad/Training_65508578.jpg  
  inflating: train/sad/Training_65516023.jpg  
  inflating: train/sad/Training_65524027.jpg  
  inflating

In [5]:
!ls -lh /content


total 61M
-rw-r--r-- 1 root root  61M Jul 19  2020 fer2013.zip
-rw-r--r-- 1 root root   70 Nov 25 15:46 kaggle.json
drwxr-xr-x 1 root root 4.0K Nov 20 14:30 sample_data
drwxr-xr-x 9 root root 4.0K Nov 25 15:46 test
drwxr-xr-x 9 root root 4.0K Nov 25 15:46 train


In [6]:
%%writefile data_loader.py
import tensorflow as tf

def load_fer_image_dataset(train_dir="/content/train", test_dir="/content/test"):
    img_size = 48
    batch_size = 64

    train_ds = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True
    ).flow_from_directory(
        train_dir,
        target_size=(img_size, img_size),
        color_mode="grayscale",
        class_mode="categorical",
        batch_size=batch_size
    )

    test_ds = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
    ).flow_from_directory(
        test_dir,
        target_size=(img_size, img_size),
        color_mode="grayscale",
        class_mode="categorical",
        batch_size=batch_size
    )

    return train_ds, test_ds


Writing data_loader.py


In [7]:
%%writefile main.py
from data_loader import load_fer_image_dataset
from model_builder import build_emotion_cnn
from trainer import train_model

if __name__ == "__main__":
    print("Loading FER-2013 image dataset...")
    train_ds, test_ds = load_fer_image_dataset()

    print("Building model...")
    model = build_emotion_cnn()

    print("Training model...")
    history = model.fit(train_ds, epochs=40, validation_data=test_ds)

    model.save("emotion_cnn_model.h5")
    print("Model saved as emotion_cnn_model.h5")


Writing main.py


In [8]:
%%writefile data_loader.py
import tensorflow as tf

def load_fer_image_dataset(train_dir="/content/train", test_dir="/content/test"):
    img_size = 48
    batch_size = 64

    train_ds = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True
    ).flow_from_directory(
        train_dir,
        target_size=(img_size, img_size),
        color_mode="grayscale",
        class_mode="categorical",
        batch_size=batch_size
    )

    test_ds = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
    ).flow_from_directory(
        test_dir,
        target_size=(img_size, img_size),
        color_mode="grayscale",
        class_mode="categorical",
        batch_size=batch_size
    )

    return train_ds, test_ds


Overwriting data_loader.py


In [9]:
%%writefile main.py
from data_loader import load_fer_image_dataset
from model_builder import build_emotion_cnn
from trainer import train_model

if __name__ == "__main__":
    print("Loading FER-2013 image dataset...")
    train_ds, test_ds = load_fer_image_dataset()

    print("Building model...")
    model = build_emotion_cnn()

    print("Training model...")
    history = model.fit(train_ds, epochs=40, validation_data=test_ds)

    model.save("emotion_cnn_model.h5")
    print("Model saved as emotion_cnn_model.h5")


Overwriting main.py


In [10]:
%%writefile trainer.py
def train_model(model, train_ds, test_ds):
    history = model.fit(
        train_ds,
        epochs=40,
        validation_data=test_ds
    )
    model.save("emotion_cnn_model.h5")
    return history


Writing trainer.py


In [12]:
%%writefile model_builder.py
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Flatten, Dropout

def build_emotion_cnn():
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(48,48,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.25))

    # Block 2
    model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.25))

    # Block 3
    model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model


Writing model_builder.py


In [13]:
%%writefile trainer.py
def train_model(model, train_ds, test_ds):
    history = model.fit(
        train_ds,
        epochs=40,
        validation_data=test_ds
    )
    model.save("emotion_cnn_model.h5")
    return history


Overwriting trainer.py


In [14]:
!ls -l /content


total 61812
-rw-r--r-- 1 root root      873 Nov 25 15:46 data_loader.py
-rw-r--r-- 1 root root 63252113 Jul 19  2020 fer2013.zip
-rw-r--r-- 1 root root       70 Nov 25 15:46 kaggle.json
-rw-r--r-- 1 root root      502 Nov 25 15:46 main.py
-rw-r--r-- 1 root root     1408 Nov 25 15:46 model_builder.py
drwxr-xr-x 2 root root     4096 Nov 25 15:46 __pycache__
drwxr-xr-x 1 root root     4096 Nov 20 14:30 sample_data
drwxr-xr-x 9 root root     4096 Nov 25 15:46 test
drwxr-xr-x 9 root root     4096 Nov 25 15:46 train
-rw-r--r-- 1 root root      201 Nov 25 15:46 trainer.py


In [15]:
!python main.py


2025-11-25 15:46:44.167926: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764085604.187331     476 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764085604.192802     476 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764085604.206432     476 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764085604.206464     476 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764085604.206468     476 computation_placer.cc:177] computation placer alr

In [16]:
%%writefile predictor.py
import cv2
import numpy as np
from tensorflow.keras.models import load_model

EMOTIONS = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

def predict_emotion(model_path, img_path):
    model = load_model(model_path)

    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    face = cv2.resize(gray, (48, 48))
    face = face.reshape(1, 48, 48, 1) / 255.0

    preds = model.predict(face)[0]
    emotion = EMOTIONS[np.argmax(preds)]
    return emotion, preds


Writing predictor.py


In [18]:
!ls -R /content


Streaming output truncated to the last 5000 lines.
Training_31025537.jpg  Training_61965744.jpg  Training_91420489.jpg
Training_31040721.jpg  Training_61969917.jpg  Training_91425869.jpg
Training_31058874.jpg  Training_61986186.jpg  Training_91434687.jpg
Training_31068769.jpg  Training_61996410.jpg  Training_91439170.jpg
Training_31091832.jpg  Training_61999969.jpg  Training_91462241.jpg
Training_31092411.jpg  Training_62002819.jpg  Training_91468416.jpg
Training_31092560.jpg  Training_6202160.jpg   Training_91481641.jpg
Training_31093924.jpg  Training_62054212.jpg  Training_91495008.jpg
Training_31161458.jpg  Training_62055162.jpg  Training_9150638.jpg
Training_31174655.jpg  Training_62060201.jpg  Training_91511544.jpg
Training_3118501.jpg   Training_62067397.jpg  Training_91512451.jpg
Training_31202995.jpg  Training_62087948.jpg  Training_91516775.jpg
Training_31213882.jpg  Training_62089158.jpg  Training_91518682.jpg
Training_31227043.jpg  Training_62100199.jpg  Training_91519570.jp

In [28]:
from google.colab import files
uploaded = files.upload()  # This will open a file picker


Saving download.jpg to download (1).jpg


In [29]:
list(uploaded.keys())


['download (1).jpg']

In [30]:
from predictor import predict_emotion

# Replace 'download.jpg' with the actual uploaded filename
emotion, preds = predict_emotion(
    "emotion_cnn_model.h5",
    "/content/download.jpg"
)

print("Predicted Emotion:", emotion)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 682ms/step
Predicted Emotion: Happy
